# pySmartOrgAPI - A python class for interfacing with SmartOrg kirk API <a class="jp-toc-ignore"></a>

<h2>Setup</h2>

<h3>Import libraries</h3>

In [ ]:
import smartorgclass

import os
from dotenv import load_dotenv

import pandas as pd

import json
import base64
import urllib.parse

<h3>Load environmental variables from .env file</h3>

- load_dotenv() will load the .env file in the current directory.  This file should contain the password needed to authenticate to the kirk API:
``` sh
PASSWORD=<your_PNAV_password>
```

In [ ]:
load_dotenv()

<h1>Authentication & Login</h1> 

<h2>SmartOrg class _init_() function</h2>
    - Generates Hmac needed for authorization to PNAV 8.x. and then calls the getToken() method to generate initial JWT token

<b>Args:</b>
- <b>username</b> (str): 
  - String representing a registered username
- <b>password</b> (str): 
  - String with the registered user's password
- <b>server</b> (str):
  - String representing the SmartOrg server to communicate with, e.g.:
    - 'dev.smartorg.com'
    - 'trials.smartorg.com'
    - etc.
- <b>timeout</b> (float, optional):
  - How many seconds to wait for the server to send data before giving up, as a float.  Default value = 30.
- <b>verify</b> (bool, optional):
  - Boolean which controls whether we verify the server’s TLS certificate. Default value = True
 
<b>API:</b> In getToken() method
- <b>POST:</b> /wizard-api/framework/login/a/{username}

<b>Returns:</b>
- the getToken() method returns a JWT token if authentication is successful.  This JWT token is stored automatically as a variable (this.token) inside the SmartOrg class that has been created

<b>Notes:</b>
- every time a method (must be a method that passes the current JWT token to it) in the SmartOrgClass is successfully called, it will automatically update the JWT token that is stored in this class
- the JWT token is set by the software to expire 30 minutes after it is created



In [ ]:
username = 'DaveAppleConnect'
server = 'pn-smartorg.rap.apple.com'

In [ ]:
so = smartorgclass.SmartOrg(username,os.getenv('PASSWORD'),server, verify=False)

#### JWT token currently stored in the instantiated SmartOrgClass <a class="jp-toc-ignore"></a>

In [ ]:
so.token

#### What is a JWT token?<a class="jp-toc-ignore"></a>

JWT stands for "JSON Web Token." It is a compact, URL-safe means of representing claims to be transferred between two parties. The claims in a JWT are encoded as a JSON object that is used as the payload of a JSON Web Signature (JWS) structure or as the plaintext of a JSON Web Encryption (JWE) structure, enabling the claims to be digitally signed or integrity-protected with a Message Authentication Code (MAC) and/or encrypted.

#### Decoding JWT token payload<a class="jp-toc-ignore"></a>

In [ ]:
import jwt
import datetime

# Decode the JWT token
try:
    decoded = jwt.decode(so.token, options={"verify_signature": False})
    print(decoded)
    # Extract the expiration time
    exp_timestamp = decoded.get("exp")
    
    if exp_timestamp:
        # Convert the expiration time to a human-readable format
        exp_time = datetime.datetime.fromtimestamp(exp_timestamp, datetime.timezone.utc)
        print(f"Token expiration time (UTC): {exp_time}")
    else:
        print("Expiration time not found in token")
        
except jwt.ExpiredSignatureError:
    print("Token has expired")
except jwt.InvalidTokenError:
    print("Invalid token")


<h1>Portfolios/Trees</h1>

<h2>portfolios() method</h2>
- Get portfolios   

<b>Args:</b>
- <b>none</b>

<b>API:</b>
- <b>GET:</b> domain/nav/portfolios

<b>Returns:</b>
- list of top-most (root) node documents for all portfolios on the server.  These documents are saved inside the mongoDB astro_nodes collection.

In [ ]:
ports = so.portfolios()
print(ports.keys())
print()
print('i\tPortfolios (treeID)\t\tTop-most nodeID')
for i,p in enumerate(ports['portfolios']):
    print(i,'\t',p['name'],'\t\t',p['_id'])

<h3> Converting the list of portfolios to a pandas dataframe for better clarity </h3>

In [ ]:
df_ports = pd.DataFrame(ports['portfolios'])
df_ports

<h2>createPortfolio() method</h2>
- Create new portfolio

<b>Args:</b>
- <b>newPortfolioName</b>
    - string representing name of new portfolio

<b>API:</b>
- <b>POST:</b> domain/admin/portfolio/new

<b>Returns:</b>
- upon success, returns
``` sh
{'status': 0, 'message': 'Create a new portfolio', 'nodeID': <nodeID of porfolio root node>}
```

In [ ]:
newPortfolioName = 'Dave New Portfolio'
res = so.createPortfolio(newPortfolioName)
res

<h2>treeFor() method</h2>
- Get portfolio tree    

<b>Args:</b>
- <b>portfolioName</b> (str): 
  - String with name of portfolio (aka treeID)

<b>API:</b>
  - <b>GET:</b> domain/nav/tree

<b>Returns:</b>
- List of all node documents for which the 'treeID' matches the 'portfolioName' passed to this method as an argument.  Note that the nodes returned are in a random order.  To generate a proper tree, one must do some additional ordering

In [ ]:
portfolioName = 'SmartOrg Standard Demo with 40 Years'
nodes = so.treeFor(portfolioName)
nodes

<h3> Converting the list of dictionaries to a pandas dataframe for better clarity </h3>

In [ ]:
df = pd.DataFrame(nodes)
df.set_index('_id',inplace=True) # set dataframe index to node id
df['id']=df.index # create a column with node id's for convenience
df

#### The column labels for this pandas dataframe represent the keys in the database document.  Note that the index of the dataframe is the _id key. <a class="jp-toc-ignore"></a>

In [ ]:
df.columns

<h3> NodeTree class for ordering portfolio tree</h3>

In [ ]:
class NodeTree(object):
    def __init__(self, df, id = None, children = None):
        self.id = id
        self.df = df
        self.children = self.df.loc[self.id].children

    def preorder(self):
        yield self.df.loc[self.id]
        for child in self.children:
            y = NodeTree(self.df,child)
            yield from y.preorder()

#### Get top-most node in portfolio tree

In [ ]:
top_node =top_node = df[df.parent.astype(bool)==False].index[0] # find pandas row where there is no parent defined and then get the value of the first element in the row
top_node

#### Generate Portfolio Tree

In [ ]:
treeNodes = NodeTree(df,top_node)
for i, y in enumerate(treeNodes.preorder()):
    pathLength = len(y['path'])
    print('   '*pathLength,f"{y['name']} - (nodeID: {y['id']})")

<h3> Get only list of leaf nodes (no children)</h3>
- use list comprehension

In [ ]:
children = [node for node in nodes if node['children']==[]]
children

In [ ]:
for child in children:
    print(child['name'])

<h2>getSubtree() method</h2>
-Get sub-tree

<b>Args:</b>
- <b>loadNodeList</b> (List(str)): 
    - list of strings with nodeID's to load.  Nodes in this list should not be leaves.
    - Note that the list can contain multiple sub-portfolios to load the info for

<b>API:</b>
  - <b>POST:</b> domain/nav/get-subtree

<b>Returns:</b><br>
    - List of astro_node documents for sub-nodes in portfolio tree below nodes in loadNodeList

In [ ]:
treeID = 'SmartOrg Standard Demo with 40 Years'
portNodes = so.treeFor(treeID)

# Get all 2nd level nodes in portfolio which are not leaves
level=2
loadNodeList = [n['_id'] for n in portNodes if len(n['path'])==level and n['children']!=[]]
loadNodeList

In [ ]:
res = so.getSubtree(loadNodeList)
res

<h2>firstNLevelTreeFor() method</h2>
-Get first N levels of tree

<b>Args:</b>
- <b>portfolioName</b> (str): 
    - string with name of portfolio
- <b>nLevel</b> (int): 
    - number of levels of tree to return
    - NOTE:  nLevel is currently hard-codedd to 4 in back-end codebase
        
<b>API:</b>
  - <b>POST:</b> domain/nav/tree-first-n-level

<b>Returns:</b><br>
    - List of astro_node documents for first N levels of portfolio tree

In [ ]:
portfolioName = 'SmartOrg Standard Demo with 40 Years'
nLevel = 4
res = so.firstNLevelTreeFor(portfolioName,nLevel)
res

<h2>getTemplateRestrictions() method</h2>
- Get template restrictions for a portfolio

<b>Args:</b>
- <b>treeID</b> (str): 
    - string with name of portfolio

        
<b>API:</b>
  - <b>GET:</b> domain/admin/portfolio/restrict/template

<b>Returns:</b>
- Dictionary containing two keys:  'restrictedTemplates' and 'remainingTemplates' which contain list values of templates

In [ ]:
treeID = 'Demo 2021 Make Sell Portfolio'
res = so.getTemplateRestrictions(treeID)
res

<h2>setTemplateRestrictions() method</h2>
-Set template restrictions for a portfolio

<b>Args:</b>
- <b>treeID</b> (str): 
    - string with name of portfolio
- <b>chosenTemplates</b> (List[str]): 
    - list of templates as strings

        
<b>API:</b>
  - <b>PUT:</b>domain/admin/portfolio/restrict/template

<b>Returns:</b>
- Nothing

In [ ]:
chosenTemplates = ['Demo2021MakeSell','Demo2021MakeSellMatureAssessment40Years']
treeID = 'SmartOrg Standard Demo with 40 Years'
res = so.setTemplateRestrictions(treeID,chosenTemplates)
res

<h2>fetchAllExportedPortfolioPaths() method</h2>
- Fetch all exported portfolios paths

- <b>NOTE:  this is not currently enabled on Apple PNAV servers</b>


<b>Args:</b>
- <b>none</b> 
  - _id of top-most (root) node of portfolio
 
<b>Description:</b>
- Fetch list of all exported portfolios paths located at /opt/rangal/1.0.0/tmp/export on server

<b>API:</b>
  - <b>GET:</b> domain/admin/portfolio/exported

<b>Returns:</b>
- dict with 3 keys: ['status', 'message','encodedExportedPortfolioList'],  NOTE:  strings in list are url and 64-bit encoded

In [ ]:
res = so.fetchAllExportedPortfolioPaths()
res

#### Decoded portfolio paths <a class="jp-toc-ignore"></a>

In [ ]:
for i,p in enumerate(res['encodedExportedPortfolioList']):
    decoded_str = base64.b64decode(urllib.parse.unquote(p)).decode('utf-8')
    print(i,decoded_str)

<h2>exportPortfolio() method</h2>
- Export portfolio

- <b>NOTE:  this is not currently enabled on Apple PNAV servers</b>

<b>Args:</b>
- <b>nodeID</b> (str): 
  - _id of top-most (root) node of portfolio
 
<b>Description:</b>
- Exports portfolio with root node of nodeID to /opt/rangal/1.0.0/tmp/export on server

<b>API:</b>
  - <b>POST:</b> domain/admin/portfolio/export

In [ ]:
port = so.portfolios()['portfolios'][0]
print('Portfolio:  ',port['name'], 'nodeID:  ',port['_id'])
nodeID = port['_id']

res = so.exportPortfolio(nodeID)
res

<h2>importPortfolio() method</h2>
- Import portfolio from exported portfolios

- <b>NOTE:  this is not currently enabled on Apple PNAV servers</b>

<b>Args:</b>
- <b>includeData (bool):</b> 
    - boolean to indicate whether to include data in portfolio import
- <b>pathToImportedFiles64 (str):</b>
    - 64-bit encoded path to portolio files to be imported
- <b>newTree64 (str):</b>
    - encoded name of new portofolio to which to import to

<b>API:</b>
  - <b>POST:</b> domain/admin/portfolio/import

In [ ]:
newTree = "Dave's Import Test rev b"
newTree64= base64.b64encode(newTree.encode('utf-8')).decode('utf-8')
newTree64

In [ ]:
s = '/opt/rangal/1.0.0/tmp/export/New Product Introduction/20230928.105200'
pathToImportedFiles64 = base64.b64encode(s.encode('utf-8')).decode('utf-8')
pathToImportedFiles64

In [ ]:
res = so.importPortfolio(True,pathToImportedFiles64,newTree64)
res

<h1>Nodes</h1>

<h2>createNode() method</h2>
- Create a new node

<b>Args:</b>
- <b>parentNodeID</b> (str): 
  - String with parent node _id
- <b>newNodeName</b> (str): 
  - String name of new node in tree
- <b>templateName</b> (str): 
  - String with name of template to use for this new node
- <b>parentTags</b> (List[str], optional): 
  - Default value = None
- <b>newNodeTags</b> (List[str], optional): 
  - Default value = ['all']
  - Add additional category tags to the list in the form of '<categoryName>:<categoryEntry>'
- <b>newNodeDropdownTags</b> (List[str], optional): 
  - Default value = None


<b>API:</b>
  - <b>POST:</b> domain/node/doc

<b>Returns:</b>

```sh 
{'status':0, 'message': 'Created a new node', 'nodeID':'66722017bdj3923'}
```

In [ ]:
parentNodeID = '662b003555f1646870a4ea13'
newNodeName = 'My New Project'
templateName = 'Demo2021MakeSellMatureAssessment40Year'

In [ ]:
res = so.createNode(parentNodeID, newNodeName, templateName)
res

<h2>deleteNode() method</h2>
- Delete a new node

Note that deleting a node moves it to the recycle bin. 
To permanently delete a node, it must be deleting from 
the recycle bin using the permanentDeleteRecord API call
    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - String with node _id


<b>API:</b>
  - <b>DELETE:</b> domain/node/doc

<b>Returns:</b>

```sh 
{'status':0, 'message': 'Created a new node', 'nodeID':'66722017bdj3923'}
```

In [ ]:
nodeID = '66b67f8f419050f2f274db5e'
res = so.deleteNode(nodeID)
res

<h2>nodeBy() method</h2>
- Get document from astro_nodes collection for document nodeID

    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - String with node _id


<b>API:</b>
  - <b>GET:</b> domain/node/doc

<b>Returns:</b>
- dictionary containg nodeID document from astro_nodes collection

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.nodeBy(nodeID)
res

<h2>fetchInputData() method</h2>
- Fetch input data for a node

    
<b>Args:</b>
- <b>treeID</b> (str): 
  - String with treeID (name of portfolio)
- <b>nodeID</b> (str): 
  - String with nodeID


<b>API:</b>
  - <b>POST:</b> domain/node/download/input

<b>Returns:</b>


In [ ]:
treeID = 'SmartOrg Standard Demo with 40 Years'
nodeID = '662b004e130ccbf4748c7c24'
res = so.fetchInputData(treeID, nodeID)
res

<h2>nodeByPost() method</h2>
- Get node by Post

    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - String with nodeID


<b>API:</b>
  - <b>POST:</b> domain/nodeByPost

<b>Returns:</b>
    - dictionary containing docuemtg from astro_nodes for nodeID


In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.nodeByPost(nodeID)
res

<h2>getSimpleOutputs) method</h2>
- Get simple outputs for a node

    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - String with nodeID


<b>API:</b>
  - <b>GET:</b> domain/output

<b>Returns:</b>
    - dictionary with simple outputs from astro_data for nodeID


In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.getSimpleOutputs(nodeID)
res

<h2>changeLog() method</h2>
- Get change log for a node

    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - String with nodeID


<b>API:</b>
  - <b>GET:</b> domain/portfolio/changes

<b>Returns:</b>
    - List of dicts containing the node change log information

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.changeLog(nodeID)
res

<h2>getAllTemplates() method</h2> 
- Get all available templates for a node

<b>Args:</b>
- <b>node_id</b> (str): 
  - String with nodeID

<b>API:</b>
  - <b>GET:</b> domain/admin/templates/all

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.getAllTemplates(nodeID)
print(json.dumps(res,indent=2))

<h2>templatesFor() method</h2> 
- Get templates for a node
    
<b>Args:</b>
- <b>node_id</b> (str): 
  - String with nodeID

<b>API:</b>
  - <b>GET:</b> domain/templates/list

<b>Returns:</b>
- dictionary 'templates' key and list of dicts containing templates that are in the list of Chosen Templates for this node

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.templatesFor(nodeID)
res

<h2>getOrphanNodesCount() method</h2>
-Get orphanned node count for a portfolio

<b>Args:</b>
- <b>treeID</b> (str): 
  - String representing name of portfolio

<b>API:</b>
  - <b>GET:</b> domain/portfolio/orphan

<b>Returns:</b>

- number of orphanned noes in the selected portfolio

In [ ]:
treeID = 'SmartOrg Standard Demo with 40 Years'
res = so.getOrphanNodesCount(treeID)
print(json.dumps(res,indent=2))

<h1>Action Menu</h1>

<h2>actionMenuFor() method</h2>
- Get action menu for node
    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - string representing node _id for a node

<b>API:</b>
  - <b>GET:</b> Get action menu for node

<b>Returns</b>:
    - <b>dictionary</b> with two main keys:  'tagData' and 'categoryConfig'
    - <b>'tagData'</b>  contains a list of dicts showing the category assignments for each node in the portfolio
    - <b>'categoryConfig'</b> retunrns a list of dicts showing all the categories, category settings and category entries

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.actionMenuFor(nodeID)
res

In [ ]:
for a in res['menuItems']['Actions']:
    if 'dropdown' in a:
        print(a['name'])
        for b in a['dropdown']:
            print(f'\t {b['name']}, {b['Command']}')

<h2>actionFor() method</h2>
- Get display name list for a node and its template
    
<b>Args:</b>
- <b>actionID</b> (str):
    - string with name of action in action menu 
- <b>nodeID</b> (str): 
  - string representing node _id for a node
- <b>packedReportOptions</b> (str, optional): 
  - string with 64-bit encoded report options dict
  - Default value:  "e30=",  decoded value:  {}
- <b>packedExcludeFilterOptions</b> (str, optional): 
  - string with 64-bit encoded report options dict
  - Default value:  "W10=",  decoded value:  []

<b>API:</b>
  - <b>POST:</b> template/actionMenu
    
<b>Returns</b>:
    - dictionary containing the following keys:  
    - ['data'] -> with following sub-keys:
    - ['inputScreenConfig', 'selectedInputKey', 'settings'] where ['inputScreenConfig'] has the following sub-keys:
    - ['main', 'sibling', 'user', 'nodeAttribute'] where ['main'] sub-keys depend on command in action menu

<h3>Example using input table</h3>

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
actionID = 'INPUTTABLE:DevelopmentCostsandTiming'
res = so.actionFor(actionID, nodeID)
res

<h3>Example using input screen</h3>

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
actionID = 'INPUTS:ProjectLifeCycle'
res = so.actionFor(actionID, nodeID)
res

<h3>Example using tornado</h3>

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
actionID = 'Tornado'
res = so.actionFor(actionID, nodeID)
res

<h1>Categories/Tags</h1>

<h2>getAssignCategory() method</h2>
- Get categories and category assignments for a portfolio
    
<b>Args:</b>
- <b>treeID</b> (str): 
  - string representing name of a portfolio

<b>API:</b>
  - <b>POST:</b> domain/category/assign/display

<b>Returns</b>:
    - <b>dictionary</b> with two main keys:  'tagData' and 'categoryConfig'
    - <b>'tagData'</b>  contains a list of dicts showing the category assignments for each node in the portfolio
    - <b>'categoryConfig'</b> retunrns a list of dicts showing all the categories, category settings and category entries

In [ ]:
treeID = 'SmartOrg Standard Demo with 40 Years'
res = so.getAssignCategory(treeID)
res

<h2>arrangeCategoriesConfig() method</h2>
- Arrange order of categories
    
<b>Args:</b>
- <b>rootNodeID</b> (str): 
  - string with astro_node _id of portfolio root node
- <b>categoriesConfig</b> (List[dict]):
    - list of category dicts in order of arrangement

<b>API:</b>
  - <b>POST:</b> domain/category/config/arrange

<b>Returns:</b>
    = True or False

In [ ]:
categoriesConfig = [
    {
        "CategoryName": "LineOfBusiness",
        "AppliesTo": "Leaves",
        "IsMultiSelect": "false",
        "ShowOnCategory": True,
        "AdminEditing": True,
        "ContributorEditing": False,
        "AutoPropagateUp": "true",
        "CategoryEntries": [
            "ConsumerProducts",
            "HouseholdAppliances",
            "SportingGoods"
        ],
        "SortEntries": None
    },
    {
        "CategoryName": "ProjectHealth",
        "AppliesTo": "Leaves",
        "IsMultiSelect": "false",
        "ShowOnCategory": True,
        "AdminEditing": True,
        "ContributorEditing": False,
        "AutoPropagateUp": "true",
        "CategoryEntries": [
            "Green",
            "Yellow",
            "Red"
        ],
        "SortEntries": None
    },
    {
        "CategoryName": "TestOne",
        "AppliesTo": "Leaves",
        "IsMultiSelect": "false",
        "ShowOnCategory": True,
        "AdminEditing": True,
        "ContributorEditing": False,
        "AutoPropagateUp": "true",
        "CategoryEntries": [
            "EntryA",
            "EntryB"
        ],
        "SortEntries": None
    },
    {
        "CategoryName": "TestTwo",
        "AppliesTo": "Leaves",
        "IsMultiSelect": "false",
        "ShowOnCategory": True,
        "AdminEditing": True,
        "ContributorEditing": False,
        "AutoPropagateUp": "true",
        "CategoryEntries": [
            "EntryA",
            "EntryB"
        ],
        "SortEntries": None
    },
    {
        "CategoryName": "Country",
        "AppliesTo": "Leaves",
        "IsMultiSelect": "true",
        "ShowOnCategory": False,
        "AdminEditing": False,
        "ContributorEditing": False,
        "AutoPropagateUp": "true",
        "CategoryEntries": [
            "France",
            "Germany",
            "Netherlands"
        ],
        "SortEntries": None
    }
]

rootNodeID = '662affe27ada4f0369eee3d3'

In [ ]:
res = so.arrangeCategoriesConfig(rootNodeID, categoriesConfig)
res

<h2>deleteCategoryConfig() method</h2>
- Delete category
    
<b>Args:</b>
- <b>rootNodeID</b> (str): 
  - string with astro_node _id of portfolio root node
- <b>categoryName</b> (str):
    - string with name of category to delete

<b>API:</b>
  - <b>DELETE:</b> string with name of category to delete

<b>Returns:</b>
    - True or False

In [ ]:
categoryName = 'TestTwo'
rootNodeID = '662affe27ada4f0369eee3d3'

In [ ]:
res = so.deleteCategoryConfig(rootNodeID,categoryName)
res

<h2>categoryConfigFor() method</h2>
- Get categories for a portfolio
    
<b>Args:</b>
- <b>rootNodeID</b> (str): 
  - string with astro_node _id of portfolio root node

<b>API:</b>
  - <b>GET:</b> domain/category/config/list
        Returns:

<b>Returns:</b>
    - returns a list of dicts showing all the categories, category settings and category entries

In [ ]:
rootNodeID = '662affe27ada4f0369eee3d3'
res = so.categoryConfigFor(rootNodeID)
res

<h2>saveCategoryConfig() method</h2>
- Save a new category or update an existing category configuration
    
<b>Args:</b>
- <b>rootNodeID</b> (str): 
  - string with astro_node _id of portfolio root node
- <b>categoryConfig</b> (dict):
    - (see below)
- <b>renameEntriesTracker</b> (List[dict]):
    - [{'entry': <entry_name>, 'state': False, 'vals': None, 'isDuplicate':False}]
- <b>changedCategoryName</b> (dict, optional):
    - default value = {}

<b>API:</b>
  - <b>POST:</b> domain/category/config/save

<b>Returns:</b>
    - True or False

In [ ]:
categoryConfig = {
    "CategoryName": "TestTwo",
    "AppliesTo": "Leaves",
    "IsMultiSelect": "false",
    "ShowOnCategory": True,
    "AdminEditing": True,
    "ContributorEditing": False,
    "AutoPropagateUp": "true",
    "CategoryEntries": [
        "EntryA",
        "EntryB"
    ],
    "SortEntries": None
}

renameEntriesTracker = [
    {
        "entry": "EntryA",
        "state": False,
        "vals": None,
        "isDuplicate": False
    }
]

rootNodeID = '662affe27ada4f0369eee3d3'

In [ ]:
res = so.saveCategoryConfig(rootNodeID,categoryConfig,renameEntriesTracker)
res

<h2>tagsFor() method</h2>
- Get tags (categories) for a node
    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - string representing node _id for a node
- <b>filter</b> (str,optional):
    - string containing a filters
    - default value = ''

<b>API:</b>
  - <b>GET:</b>  domain/tags

<b>Returns:</b>
    - list of tags


In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.tagsFor(nodeID)
res

<h2>dropdownTagsFor() method</h2>
- Get dropdown tags for node
    
<b>Args:</b>
- <b>nodeID</b> (str): 
  - string representing node _id for a node
- <b>filter</b> (str,optional):
    - NOT IMPLEMENTED in this wrapper - string representing tree-filter selections

<b>API:</b>
  - <b>GET:</b>  domain/dropdownTags

<b>Returns:</b>
    - returns list of strings containing dropdown tags for node in format:  'dropdownInputName:url-encoded(64-bit encoded(value))'


In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
res = so.dropdownTagsFor(nodeID)
res

In [ ]:
for t in res:
    k,v = t.split(':')
    print(k,'\t'*3,base64.b64decode(urllib.parse.unquote(v)).decode('utf-8'))

<h1>Universal Tables</h1>

<h2>getDisplayNameList() method</h2>
-Get display name list for a node and its template

<b>Args:</b>
- <b>templateName</b> (str):
    - string with name of template
- <b>leafOrPlatform</b> (str):
    - string with either "leaf" or "platform"
- <b>nodeID</b> (str):
    - string with nodeID of node

                                   
<b>API:</b>
  - <b>POST:</b> domain/universal/io/schema/fields

<b>Returns:</b>
    -  dictionary containing the following keys:  
```sh 
        ['inputs', 'outputs', 'tableInputs', 'tableOutputs', 'categories']
        
    - Under each sub-key is a list of dicts with a json config for each of the inputs, outputs, etc.

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
templateName = 'Demo2021MakeSellMatureAssessment40Year'
res = so.getDisplayNameList(templateName,'leaf',nodeID)
res

<h2>getFieldList() method</h2>
-Get field list for universal table for a node

<b>Args:</b>
- <b>nodeID</b> (str):
    - string with nodeID of node
- <b>schema</b> (dict):
    - see example below. 
- <b>packedReportOptions</b> (str,optional):
    - string with 64-bit encoded report options dict
    - Default value:  "e30=",  decoded value:  {}
- <b>packedExcludeFilterOptions</b> (str, optional):
    - string with 64-bit encoded report options dict
    - Default value:  "W10=",  decoded value:  []

                                   
<b>API:</b>
  - <b>POST:</b> domain/universal/io/fields

<b>Returns:</b>
- dict with all the values requested in the schema for all leaves at or below nodeID

In [ ]:
nodeID = '662b004e130ccbf4748c7c24'
schema={            
    "templates": {
        "Demo2021MakeSellMatureAssessment40Year": {
            "templateName": "Demo2021MakeSellMatureAssessment40Year",
            "leaf": {
                "inputKeys": [
                    "tam",
                    "rampDuration"
                ],
                "inputTables": {},
                "outputKeys":[
                    "npvOperatingProfit"
                ],
                "outputTables": {},
                "categories": ["Country","LineOfBusiness"]
            }
        }
    }
}


In [ ]:
res = so.getFieldList(nodeID,schema)
res


<h2>saveUnivSchemaToRoot() method</h2>
-Save universal table schema to root (top-node)

<b>Args:</b>
- <b>nodeID</b> (str):
    - string with nodeID of node
- <b>schema</b> (dict):
    - description of schema?
- <b>oldName</b> (str,optional):
    - string with old name of schema to be replaced
    - Default value = None
    
                                   
<b>API:</b>
  - <b>POST:</b> domain/universal/io/schema/save

<b>Returns:</b>
    - Example of return if successful:
```sh 
{'status': 0, 'message': "Universal table 'davetestnew2' is saved", 'schemaName': 'davetestnew2'}

<h2>universalSaveInputs() method</h2>
-Save universal table schema to root (top-node)

<b>Args:</b>
- <b>treeID</b> (str):
    - string with name of portolio
- <b>inputs</b> (List(dict)):
    - example list:
```sh
exampleInputs = [
                {
                    "id": "61018b4ba2f47065daefeba1_20210827.135659_20230928.105045",
                    "details": {
                        "nodeName": "Oven",
                        "inputs": {
                            "tam": {
                                "0": "90000",
                                "1": "12500"
                            }
                        },
                        "categories": {}
                    }
                }
            ]
```

- <b>comment</b> (str):
    - string containing comment about change/update
- <b>menuID</b> (str):
    -string in the form of:  "UniversalIO:"+schemaName
    
                                   
<b>API:</b>
  - <b>POST:</b> domain/universal/io/table/save

<b>Returns:</b>
    - Example of return if successful:
```sh 
{'status': 0, 'msg': 'Universal table data saved!', 'failureList': []}

<h1>Goal Analysis</h1>

<h2>performGoalAnalysis() method</h2>
-Perform goal analysis

<b>Args:</b>
- <b>nodeID: (str)</b>
    - string representing node _id for a node
- <b>packedRangeInfo: (str)</b>
    - string with 64-bit encoded range info dictionary
    - Example of unencoded dict:
        - { "analyzeOn":"value","lowerBound":0.05,"upperBound":0.08}
        - where "analyzeOn" is either "value" or "prob"
- <b>packedMenuInfo: (str)</b>
    - string with 64-bit encoded menu info dictionary
    - Example of unencoded dict (NOTE: found in "Command": "GOAL_ANALYSIS" in template):
    - {"RollupKeys: ["grossMarginBaseYearPlus3","grossMarginBaseYearPlus5","grossMarginBaseYearPlus10","grossRevenueBaseYearPlus3","grossRevenueBaseYearPlus5","grossRevenueBaseYearPlus10"],
    - "Source":"GrossMargin_RevenueFullDistribution","MVSType":"MVSFromFittedPoints"}
- <b>packedReportOptions: (str,optional)</b>
    - 64-bit encoded report options dict
    - default value = "e30=", decoded value = "{}"
- <b>packedExcludeFilterTags: (str,optional)</b>
    - default value = "W10="
- <b>packedExcludeFilterTags: (str,optional)</b>
    - 64-bit encoded exclude filter tags
    - default value = "W10="
- <b>actionID: (str, optional):</b>
    - string representing ID of action menu 
    - default value = None 

In [ ]:
nodeID = '610094ce9104c64d28898f6f_20210827.135659_20230928.105045'

<h3>Encoding MenuInfo for performGoalAnalysis()</h3>

In [ ]:
#  MenuInfo comes directly from the template "Command":"GOAL_ANALYSIS" portfolio structure JSON 
MenuInfo = {
    "RollupKeys":
        ["grossMarginBaseYearPlus3",
         "grossMarginBaseYearPlus5",
         "grossMarginBaseYearPlus10",
         "grossRevenueBaseYearPlus3",
         "grossRevenueBaseYearPlus5",
         "grossRevenueBaseYearPlus10"
        ],
    "Source":"GrossMargin_RevenueFullDistribution",
    "MVSType":"MVSFromFittedPoints"
}

packedMenuInfo = base64.b64encode(json.dumps(MenuInfo).encode('utf-8')).decode('utf-8')
packedMenuInfo

<h3>Encoding RangeInfo for performGoalAnalysis()</h3>

In [ ]:
RangeInfo = {
    "analyzeOn":"prob",
    "lowerBound":0.85,
    "upperBound":0.95
}

packedRangeInfo = base64.b64encode(json.dumps(RangeInfo).encode('utf-8')).decode('utf-8')
packedRangeInfo

In [ ]:
nodeID = '662b003555f1646870a4ea13'
res = so.performGoalAnalysis(nodeID, packedRangeInfo,packedMenuInfo)

In [ ]:
res.keys()

In [ ]:
res['data'].keys()

In [ ]:
res['data']['data'].keys()

In [ ]:
res['data']['data']['RollupKeys']

In [ ]:
res['data']['data']['Source']

In [ ]:
res['data']['data']['MVSType']

In [ ]:
res['data']['data']['rangeInfo']

In [ ]:
for i,d in enumerate(res['data']['data']['DataForEachKey']):
    print(i,d['Key'],d['Title'],d.keys())

In [ ]:
res['data']['data']['DataForEachKey'][2]['analysisData']

In [ ]:
df_goal = pd.DataFrame(res['data']['data']['DataForEachKey'][1]['analysisData']['mainRows'])
df_goal

<h1>Users</h1>

<h2>getAllUsers() method</h2>
-Save broadcast messages

<b>Description:</b>
- Message documents are saved in the astro_messages collection of the mongoDB database

<b>Args:</b>
- <b>none</b> 

<b>API:</b>
  - <b>GET:</b> /framework/admin/user/list

<b>Returns:</b>
- List of dicts containing astro_users documents from database

In [ ]:
res = so.getAllUsers()
res

<h2>getListOfUsers() method</h2> 
- Get list of users

<b>API:</b>
  - <b>GET:</b> framework/admin/user/list

In [ ]:
res=so.getListOfUsers()
pd.DataFrame(res)

<h2>updateUserAdminSettings() method</h2> 
- Update user admin settings

<b>Args:</b>
- <b>userID</b>: (str)
    - string with userID
- <b>adminSettings</b>: (dict)
    - dictionary with keys:
        - passwordChange: (bool)  #Force user to change password after next login
        - resetToFirstLogin: (bool) #Reset user to first login state 
    

<b>API:</b>
  - <b>GET:</b> framework/admin/user/list

In [ ]:
userID = '669fdecbc46a7e8376cec1e8'
res=so.updateUserAdminSettings(userID,{'passwordChange':True,'resetToFirstLogin':True})
res

<h1>Groups</h1>

<h2>getListOfGroups() method</h2> 
- Get list of groups

<b>Args:</b>
- <b>none</b>

<b>API:</b>
  - <b>GET:</b> framework/admin/group/list

In [ ]:
res=so.getListOfGroups()
pd.DataFrame(res)

<h2>groupAddUser() method</h2> 
- Add a user to a group

<b>Args:</b>
- <b>groupID</b>: (str)
    - string with groupID
- <b>userID</b>: (str)
    - string with userID

<b>API:</b>
  - <b>POST:</b> framework/admin/user/list

In [ ]:
groupID = '669eea3d236226878fc17b58'
so.groupAddUser(groupID,userID)

<h2>groupRemoveUser() method</h2> 
- Remove a user to a group

<b>Args:</b>
- <b>groupID</b>: (str)
    - string with groupID
- <b>userID</b>: (str)
    - string with userID

<b>API:</b>
  - <b>POST:</b> framework/admin/user/list

In [ ]:
res=so.groupRemoveUser(groupID,userID)
res

<h2>getGroupRestrictions() method</h2> 
- Get group restrictions for a portfolio

<b>Args:</b>
- <b>treeID</b>: (str)
    - string representing name of portfolio

<b>API:</b>
  - <b>GET:</b> domain/admin/portfolio/restrict/group

<b>Returns:</b>
- Dictionary containing two keys:  'restrictedGroups' and 'remainingGroups' which contain list values of group names

In [ ]:
treeID = 'Demo 2021 Make Sell Portfolio'
res = so.getGroupRestrictions(treeID)
print(json.dumps(res,indent=2))

<h1>Messaging/Communications</h1>

<h2>saveMessages() method</h2>
-Save broadcast messages

<b>Description:</b>
- Message documents are saved in the astro_messages collection of the mongoDB database

<b>Args:</b>
- <b>messages</b> 

<b>API:</b>
  - <b>POST:</b> framework/admin/broadcast/messages/save

<b>Returns:</b>
- list of dicts with broadcast messages saved to the astro_messages collection including the document _id

In [ ]:
messages = [
    {
        "_id": "",
        "title": "API message saving new",
        "message": "I am trying to save this new message using the API",
        "everyoneCanSee": True,
        "alwaysShowMessage": True,
        "groups": []
    }
]

In [ ]:
res = so.saveMessages(messages)

In [ ]:
res

<h2>getMessages() method</h2>
-Get list of broadcast messages

<b>Args:</b>
- <b>none</b> 

<b>API:</b>
  - <b>POST:</b> framework/admin/broadcast/messages/list

<b>Returns:</b>
- list of dicts with all broadcast messages

In [ ]:
so.getMessages()

<h2>deleteListOfMessages() method</h2>
-Delete list of broadcast messages

<b>Args:</b>
- <b>idList</b> (list[str])
    - list of broadcast message _ids to delete

<b>API:</b>
  - <b>POST:</b> framework/admin/broadcast/messages/delete

<b>Returns:</b>
- {'n': 1,
  'electionId': '7fffffff0000000000000015',
  'opTime': 'Timestamp(1722983857, 1)',
  'ok': 1.0,
  '$clusterTime': "{'clusterTime': Timestamp(1722983857, 1), 'signature': {'hash': b'\\t\\xd3U,\\xa2@\\xa5\\xb77_\\xe1\\x9a\\xa2\\xd7\\xed\\xcf\\xab(\\x99h', 'keyId': 7351849198788018178}}",
  'operationTime': 'Timestamp(1722983857, 1)'}

In [ ]:
idList = ['66b2a11ffe1bccce3b871c8d','66b2a58b7795d57f77e8b00d']
res = so.deleteListOfMessages(idList)
res

<h2>getWelcomeMessage() method</h2>
- Get welcome message

<b>Args:</b>
- <b>messageType</b>: (str)
     -either "LICENSE" or "SECURITY_WARNING_WB"


<b>API:</b>
- <b>GET:</b> framework/admin/welcome/message

<b>Returns:</b>
>>> {'status': 0,
>>>     'data':
>>>     {
>>>       '_id':string with id of message in astro_message,
>>>       'type': 'LICENSE' or 'SECURITY_WARNING_WB',
>>>       'config':
>>>       {
>>>          'state':0,
>>>          'message': url-encoded(64-bit encoded text message)
>>>       }
>>>     }
>>> }

In [ ]:
so.getWelcomeMessage('LICENSE')

In [ ]:
urllib.parse.unquote(base64.b64decode('JTNDcCUzRVRoaXMlMjBpcyUyMHRoZSUyMGxpY2Vuc2UlMjBhZ3JlZW1lbnQuJTNDJTJGcCUzRQ=='.encode('utf-8')).decode('utf-8'))

<h2>setWelcomeMessage() method</h2>
- Set welcome message

<b>Args:</b>
- <b>messageType</b>: (str)
     -either "LICENSE" or "SECURITY_WARNING_WB"
- <b>message</b>: (str)
     -base64-encoded(url-enccoded(<html message>))
- <b>state</b>: (int)
    - 0 - Do not show
    - 1 - Show if the license is NOT accepted
    - 2 - Show on every login


<b>API:</b>
- <b>GET:</b> framework/admin/welcome/message

<b>Returns:</b>
> {'status': 0, 'messages': ['Message saved']}

In [ ]:
unencoded_message = '<p>This is the new security message</p>'
#URL- and base64-encoding of message
message = base64.b64encode(urllib.parse.quote('<p>This is the new security message</p>').encode('utf-8')).decode('utf-8')
message

In [ ]:
res = so.setWelcomeMessage("SECURITY_WARNING_WB",message,1)
res

<h1>Miscellaneous</h1>

<h2>fetchPypeerLog() method</h2>
- Fetch pypeer logs

<b>Args:</b>
- <b>daysFromToday</b>: (int)
     - days from today to retrieve pypeer logs
- <b>startTime</b>: (optional)
    - default value = None
- <b>endTime</b>: (optional)
    - default value = None
- <b>userName</b>: (str, optional)
    - default value = None
- <b>logType</b>: (optional)
    - default value = None 


<b>API:</b>
- <b>GET:</b> framework/admin/pypeer/log

In [ ]:
res = so.fetchPypeerLog(3,None,None,None)
res.keys()

In [ ]:
print('status:',res['status'],', message:',res['message'])

In [ ]:
res['logDict']

In [ ]:
pyPeerLog = base64.b64decode(urllib.parse.unquote(res['EncodedLog'])).decode('utf-8')

json.loads(pyPeerLog)

<h2>getApiVersionNumber() method</h2>
- Get API version number

<b>Args:</b>
- none


<b>API:</b>
- <b>GET:</b> framework/admin/welcome/message

<b>Returns:
- {'controllerVersion': '5.15.0', 'calcEngineMessage': {'tooOld': False}, 'monoMessage': {'tooOld': False}}
- <b>Note:</b> This API doesn't return a JWT token as it can be called without authenticating

In [ ]:
res = so.getApiVersionNumber()
res

<h2>getCalculationEngineInfo() method</h2>
- Get calculation engine info

<b>Args:</b>
- none


<b>API:</b>
- <b>GET:</b> framework/calcengine/version

<b>Returns:
- {'versionNumber': '2.3.3', 'logLevel': 'error'}
- <b>Note:</b> This API doesn't return a JWT token as it can be called without authenticating

In [ ]:
res = so.getCalculationEngineInfo()
res

<h2>getServerConfig() method</h2>
- Get server configuration

<b>Args:</b>
- none


<b>API:</b>
- <b>GET:</b> framework/config

<b>Returns:
- {'defaultAuth': 'PSW',
 'apiVersion': {'controllerVersion': '5.15.0',
  'calcEngineMessage': {'tooOld': False},
  'monoMessage': {'tooOld': False}},
 'clientAdminEmail': 'support@smartorg.com',
 'zendeskToggle': True,
 'downloadModelToggle': True,
 'richTextBox': None,
 'disableTutorial': False,
 'isInav': True,
 'calcEngineAccess': False,
 'userPortfolioAccess': False,
 'wizardUserAccess': False,
 'isCorteva': False}
- <b>Note:</b> This API doesn't return a JWT token as it can be called without authenticating

In [ ]:
res = so.getServerConfig()
res

<h2>getServerDateTime() method</h2>
- Get server date and time

<b>Args:</b>
- none


<b>API:</b>
- <b>GET:</b> framework/datetime

<b>Returns:
- 'Aug 06 2024 21:30:58 (UTC)'
- <b>Note:</b> This API doesn't return a JWT token as it can be called without authenticating

In [ ]:
res = so.getServerDateTime()
res